In [ ]:
import pandas as pd
from modsim import *
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# install Pint if necessary
try:
  from pint import UnitRegistry
except ImportError:
  !pip install pint

In [ ]:
# download modsim.py if necessary

from os.path import basename, exists


def download(url):
  filename = basename(url)
  if not exists(filename):
    from urllib.request import urlretrieve
    local, _ = urlretrieve(url, filename)
    print('Downloaded ' + local)


download('https://github.com/AllenDowney/ModSimPy/raw/master/' +
         'modsim.py')

In [ ]:
# Menghapus kolom yang tidak diperlukan
data_have_index = [
  "sample/anxiety-disorders-males-vs-females.csv",
  "sample/bipolar-disorder-in-males-vs-females.csv",
  "sample/depression-males-vs-females.csv",
  "sample/eating-disorders-in-males-vs-females.csv",
  "sample/Mental health Depression disorder Data.csv",
  "sample/schizophrenia-in-males-vs-females.csv"
]

for file_path in data_have_index:
  df = pd.read_csv(file_path)
  if 'index' in df.columns:
    df = df.drop('index', axis=1)
  df.to_csv(file_path, index=False)

In [ ]:
df_grouped_bundir = pd.read_csv("sample/suicide-rates-among-young-people.csv")
df_grouped_bundir = df_grouped_bundir.groupby('Year').mean().reset_index()

In [ ]:
df_grouped_bundir.columns

In [ ]:
df_grouped_bundir['Change_15_19'] = df_grouped_bundir[
  'Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS15-19']
df_grouped_bundir['Change_20_24'] = df_grouped_bundir[
  'Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS20-24']
df_grouped_bundir['Change_25_34'] = df_grouped_bundir[
  'Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS25-34']

df_grouped_bundir['Percentage_Bundir'] = ((df_grouped_bundir[
                                             'Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS15-19'] +
                                           df_grouped_bundir[
                                             'Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS20-24'] +
                                           df_grouped_bundir[
                                             'Death rate per 100,000 people - Cause: Self-harm - Sex: Both sexes - Age group: YEARS25-34']) / 300000) * 100

print(f"Minimal: {df_grouped_bundir[['Percentage_Bundir']].min()}")
print(f"Maximal: {df_grouped_bundir[['Percentage_Bundir']].max()}")

In [ ]:
# Anxiety
df_anxiety = pd.read_csv("sample/anxiety-disorders-males-vs-females.csv")
df_anxiety.isnull().sum()

In [ ]:
df_anxiety = df_anxiety.fillna(method='ffill')
df_anxiety.isnull().sum()

In [ ]:
df_anxiety = df_anxiety.rename(
  columns={'Prevalence - Anxiety disorders - Sex: Female - Age: Age-standardized (Percent)': 'Anxiety_Female',
           'Prevalence - Anxiety disorders - Sex: Male - Age: Age-standardized (Percent)': 'Anxiety_Male'})

df_anxiety_filtered = df_anxiety[(df_anxiety['Year'] >= 1990) & (df_anxiety['Year'] <= 2017)]

df_anxiety_mean = df_anxiety_filtered.groupby('Year').agg({
  'Anxiety_Male': 'mean',
  'Anxiety_Female': 'mean'
})

total_df_anxiety_mean = df_anxiety_mean['Anxiety_Female'] + df_anxiety_mean['Anxiety_Male']

total_df_anxiety_mean

In [ ]:
#Bipolar
df_bipolar = pd.read_csv("sample/bipolar-disorder-in-males-vs-females.csv")
df_bipolar.isnull().sum()

In [ ]:
df_bipolar = df_bipolar.fillna(method='ffill')
df_bipolar.isnull().sum()

In [ ]:
df_bipolar = df_bipolar.rename(
  columns={'Prevalence - Bipolar disorder - Sex: Male - Age: Age-standardized (Percent)': 'Bipolar_Male',
           'Prevalence - Bipolar disorder - Sex: Female - Age: Age-standardized (Percent)': 'Bipolar_Female'})

df_bipolar_filtered = df_bipolar[(df_bipolar['Year'] >= 1990) & (df_bipolar['Year'] <= 2017)]

df_bipolar_mean = df_bipolar_filtered.groupby('Year').agg({
  'Bipolar_Male': 'mean',
  'Bipolar_Female': 'mean'
})

total_df_bipolar_mean = df_bipolar_mean['Bipolar_Male'] + df_bipolar_mean['Bipolar_Female']

total_df_bipolar_mean

In [ ]:
#Depression
df_depression = pd.read_csv("sample/depression-males-vs-females.csv")
df_depression.isnull().sum()

In [ ]:
df_depression = df_depression.fillna(method='ffill')
df_depression.isnull().sum()

In [ ]:
df_depression = df_depression.rename(
  columns={'Prevalence - Depressive disorders - Sex: Female - Age: Age-standardized (Percent)': 'Depression_Female',
           'Prevalence - Depressive disorders - Sex: Male - Age: Age-standardized (Percent)': 'Depression_Male'})

df_depression_filtered = df_depression[(df_depression['Year'] >= 1990) & (df_depression['Year'] <= 2017)]

df_depression_mean = df_depression_filtered.groupby('Year').agg({
  'Depression_Male': 'mean',
  'Depression_Female': 'mean'
})

total_df_depression_mean = df_depression_mean['Depression_Male'] + df_depression_mean['Depression_Female']

total_df_depression_mean

In [ ]:
# Eating
df_eating = pd.read_csv("sample/eating-disorders-in-males-vs-females.csv")
df_eating.isnull().sum()

In [ ]:
df_eating = df_eating.fillna(method='ffill')
df_eating.isnull().sum()

In [ ]:
df_eating = df_eating.rename(
  columns={'Prevalence - Eating disorders - Sex: Male - Age: Age-standardized (Percent)': 'Eating_Male',
           'Prevalence - Eating disorders - Sex: Female - Age: Age-standardized (Percent)': 'Eating_Female'})

df_eating_filtered = df_eating[(df_eating['Year'] >= 1990) & (df_eating['Year'] <= 2017)]

df_eating_mean = df_eating_filtered.groupby('Year').agg({
  'Eating_Male': 'mean',
  'Eating_Female': 'mean'
})

total_df_eating_mean = df_eating_mean['Eating_Male'] + df_eating_mean['Eating_Female']

total_df_eating_mean

In [ ]:
# Schizo
df_schizo = pd.read_csv('sample/schizophrenia-in-males-vs-females.csv')
df_schizo.isnull().sum()

In [ ]:
df_schizo = df_schizo.fillna(method='ffill')
df_schizo.isnull().sum()

In [ ]:
df_schizo = df_schizo.rename(
  columns={'Prevalence - Schizophrenia - Sex: Female - Age: Age-standardized (Percent)': 'Schizo_Female',
           'Prevalence - Schizophrenia - Sex: Male - Age: Age-standardized (Percent)': 'Schizo_Male'})

df_schizo_filtered = df_schizo[(df_schizo['Year'] >= 1990) & (df_schizo['Year'] <= 2017)]

df_schizo_mean = df_schizo_filtered.groupby('Year').agg({
  'Schizo_Male': 'mean',
  'Schizo_Female': 'mean'
})

total_df_schizo_mean = df_schizo_mean['Schizo_Male'] + df_schizo_mean['Schizo_Female']

total_df_schizo_mean

In [ ]:
# Monte Carlo simulation

'''
dictionary yang memetakan setiap penyakit mental ke rentang prevalensinya (dalam persentase) untuk laki-laki dan perempuan.
Nilai-nilai ini diambil dari data diatas.
'''
prevalence_range = {
  'schizophrenia': {'male': (total_df_schizo_mean.min(), total_df_schizo_mean.max()),
                    'female': (total_df_schizo_mean.min(), total_df_schizo_mean.max())},
  'anxiety': {'male': (total_df_anxiety_mean.min(), total_df_anxiety_mean.max()),
              'female': (total_df_anxiety_mean.min(), total_df_anxiety_mean.max())},
  'bipolar': {'male': (total_df_bipolar_mean.min(), total_df_bipolar_mean.max()),
              'female': (total_df_bipolar_mean.min(), total_df_bipolar_mean.max())},
  'eating_disorders': {'male': (total_df_eating_mean.min(), total_df_eating_mean.max()),
                       'female': (total_df_eating_mean.min(), total_df_eating_mean.max())},
  'depression': {'male': (total_df_depression_mean.min(), total_df_depression_mean.max()),
                 'female': (total_df_depression_mean.min(), total_df_depression_mean.max())},
}

'''
dictionary yang memetakan setiap penyakit mental ke probabilitas bunuh diri yang terkait
'''
suicide_risk = {
  'schizophrenia': 0.1,
  'anxiety': 0.05,
  'bipolar': 0.08,
  'eating_disorders': 0.02,
  'depression': 0.07,
}

def run_simulation(pop_male, pop_female, prevalence_range, suicide_risk, num_points=10):
  total_suicide_male = 0
  total_suicide_female = 0

  # Simulasi untuk laki-laki
  for disease in prevalence_range.keys():
    min_val, max_val = prevalence_range[disease]['male']
    preval_values = np.linspace(min_val, max_val, num_points)
    num_male_with_disease = int(pop_male * np.random.choice(preval_values) / 100)
    num_suicide_male = np.random.binomial(num_male_with_disease, suicide_risk[disease])
    total_suicide_male += num_suicide_male

  # Simulasi untuk perempuan
  for disease in prevalence_range.keys():
    min_val, max_val = prevalence_range[disease]['female']
    preval_values = np.linspace(min_val, max_val, num_points)
    num_female_with_disease = int(pop_female * np.random.choice(preval_values) / 100)
    num_suicide_female = np.random.binomial(num_female_with_disease, suicide_risk[disease])
    total_suicide_female += num_suicide_female

  return total_suicide_male, total_suicide_female


'''
list dari tuple, di mana setiap tuple berisi jumlah populasi laki-laki dan perempuan yang akan digunakan dalam simulasi.
'''
populations = [(200000, 350000), (1239876, 1304020), (22009, 40210), (30000, 12000)]
iterations = 1000   # jumlah iterasi yang akan dilakukan dalam simulasi Monte Carlo.

# Menjalankan simulasi
for pop_male, pop_female in populations:
  results_male = []
  results_female = []
  for _ in range(iterations):
    suicide_male, suicide_female = run_simulation(pop_male, pop_female, prevalence_range, suicide_risk)
    results_male.append(suicide_male)
    results_female.append(suicide_female)

  # Plotting hasil simulasi
  plt.figure(figsize=(14, 7))
  plt.hist(results_male, bins=30, alpha=0.7, label='Male Suicides')
  plt.hist(results_female, bins=30, alpha=0.7, label='Female Suicides')
  plt.xlabel('Number of Suicides')
  plt.ylabel('Frequency')
  plt.title(
    f'Monte Carlo Simulation of Suicide Cases Due to Mental Health Issues (Population: {pop_male} males, {pop_female} females)')
  plt.legend()
  plt.show()

  # Summary statistics
  mean_male = np.mean(results_male)
  mean_female = np.mean(results_female)
  print(f'Average suicides in males: {mean_male}')
  print(f'Average suicides in females: {mean_female}')
